In [127]:
import pandas as pd

### load table countrycode - countrycoords and preprocess

In [128]:
df_co_from_cc = pd.read_csv('./data/location_ref/councode_councood.csv')

# reame columns
df_co_from_cc = df_co_from_cc.rename({'alpha2Code': 'COUNTRY_ALPHA2_CODE'}, axis='columns')

In [129]:
df_co_from_cc['countryCoords'] = df_co_from_cc['countryCoords'].replace("Point\(", "", regex=True).replace(" ", ", ", regex=True).replace("\)", "", regex=True)


# converting to string series 
df_co_from_cc['countryCoords']= df_co_from_cc['countryCoords'].astype(str) 

# splitting at occurrence of whitespace 
df_co_from_cc['countryCoords']= df_co_from_cc['countryCoords'].str.split(",", 1) 
  
# displaying second element from list 
df_co_from_cc['country_lat_3857']  = df_co_from_cc['countryCoords'].str.get(1) 

# displaying first element from list 
df_co_from_cc['country_long_3857']  = df_co_from_cc['countryCoords'].str.get(0)
 
    
df_co_from_cc['country_lat_long_3857'] = df_co_from_cc['country_lat_3857'] + ", " + df_co_from_cc['country_long_3857'] 
df_co_from_cc = df_co_from_cc.drop(columns=['countryCoords', 'country'])

In [130]:
df_co_from_cc.head()

,COUNTRY_ALPHA2_CODE,country_lat_3857,country_long_3857,country_lat_long_3857
0,KY,19.5,-80.5,"19.5, -80.5"
1,BW,-22.2,23.7,"-22.2, 23.7"
2,TZ,-6.307,34.854,"-6.307, 34.854"
3,GH,8.0,-0.5,"8.0, -0.5"
4,MP,16.705,145.78,"16.705, 145.78"


In [131]:
df_co_from_cc.shape

(256, 4)

### load table country names multilingual - country name - country code

In [132]:
df_c_multilingu = pd.read_csv('./data/country_names_multilingual/IP2LOCATION-COUNTRY-MULTILINGUAL.CSV')
df_c_multilingu = df_c_multilingu.drop(columns=['COUNTRY_ALPHA3_CODE', 'COUNTRY_NUMERIC_CODE'])
# make COUNTRY_NAME lowercase
df_c_multilingu['COUNTRY_NAME'] = df_c_multilingu['COUNTRY_NAME'].str.lower()

df_c_multilingu = df_c_multilingu.drop(columns='LANG')
df_c_multilingu.head()

,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_NAME
0,AFRIKAANS,AD,andorra
1,AFRIKAANS,AE,verenigde arabiese emirate
2,AFRIKAANS,AF,afganistan
3,AFRIKAANS,AG,antigua en barbuda
4,AFRIKAANS,AI,anguilla


In [133]:
#drop: re, man
df_c_multilingu[df_c_multilingu['COUNTRY_NAME']=='man']

,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_NAME
13051,NORWEGIAN BOKMAL,IM,man
13798,NORWEGIAN NYNORSK,IM,man
14047,NORWEGIAN,IM,man


In [149]:
df_c_multilingu[df_c_multilingu['COUNTRY_NAME']=='mf'] 
# yt, re, man, mf
# nga = russia

,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_NAME
19562,VIETNAMESE,MF,mf


In [135]:
df_c_multilingu = df_c_multilingu.drop(index=[13051,13798, 14047, 19609,19667,19562])

In [137]:
df_c_multilingu.shape #20169 #20166 # 20165

(20165, 3)

In [138]:
df_c_multilingu['COUNTRY_NAME'].nunique()

13389

In [139]:
# remove duplicates
df_c_multilingu = df_c_multilingu[~df_c_multilingu['COUNTRY_NAME'].duplicated()]

In [140]:
df_c_multilingu.shape

(13389, 3)

### Merge dataframes

In [141]:
# merge these 2 
df_c = df_c_multilingu.merge(df_co_from_cc, how='left', on='COUNTRY_ALPHA2_CODE',suffixes=(False, False))
df_c.shape

(13747, 6)

In [142]:
df_c.head()

,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_NAME,country_lat_3857,country_long_3857,country_lat_long_3857
0,AFRIKAANS,AD,andorra,42.558333333,1.555277777,"42.558333333, 1.555277777"
1,AFRIKAANS,AE,verenigde arabiese emirate,24.4,54.3,"24.4, 54.3"
2,AFRIKAANS,AF,afganistan,34.0,66.0,"34.0, 66.0"
3,AFRIKAANS,AG,antigua en barbuda,17.116666666,-61.85,"17.116666666, -61.85"
4,AFRIKAANS,AI,anguilla,18.227222222,-63.048888888,"18.227222222, -63.048888888"


In [143]:
df_c.shape

(13747, 6)

In [144]:
# check for dupliates in COUNTRY_NAME col
# remove duplicates cityLabel entries
df_c = df_c[~df_c['COUNTRY_NAME'].duplicated()]

In [145]:
df_c.shape

(13389, 6)

In [146]:
df_c.to_csv('./data/location_ref/country_multilingu_countrycode_countrycoord.csv')

### import cityname multilingu

In [90]:
df_d = pd.read_csv('./data/location_ref/citynamemulti_councode_cityco_citypop.csv', 
                   dtype={'cityCoords': str },index_col=0)

In [91]:
df_d.head()

,language,CITY_NAME,COUNTRY_ALPHA2_CODE,population,city_lat_3857,city_long_3857,city_lat_long_3857
0,nn,Trnava,SK,65382.0,48.37775,17.586211,"48.37775, 17.586211111"
1,pl,Trnawa,SK,65382.0,48.37775,17.586211,"48.37775, 17.586211111"
2,ps,ترناوا,SK,65382.0,48.37775,17.586211,"48.37775, 17.586211111"
4,rue,Тырнава,SK,65382.0,48.37775,17.586211,"48.37775, 17.586211111"
6,si,ට්ර්නාවා,SK,65382.0,48.37775,17.586211,"48.37775, 17.586211111"


In [92]:
df_e = df_d.merge(df_co_from_cc, how='left', on='COUNTRY_ALPHA2_CODE',suffixes=(False, False))


In [93]:
# cols = list(df_e.columns.values)
col_order = ['language',
 'CITY_NAME',
 'city_lat_3857',
 'city_long_3857',
 'city_lat_long_3857',
 'population',
 'COUNTRY_ALPHA2_CODE',
 'country_lat_3857',
 'country_long_3857',
 'country_lat_long_3857']

In [97]:
df_e = df_e[col_order]

In [98]:
# make city_name lowercase
df_e['CITY_NAME'] = df_e['CITY_NAME'].str.lower()


In [99]:
df_e[2000:2100]

,language,CITY_NAME,city_lat_3857,city_long_3857,city_lat_long_3857,population,COUNTRY_ALPHA2_CODE,country_lat_3857,country_long_3857,country_lat_long_3857
2000,sa,ईरोड,11.340833,77.717222,"11.340833333, 77.717222222",151184.0,IN,22.8,83.0,"22.8, 83.0"
2001,si,", එරොදේ",11.340833,77.717222,"11.340833333, 77.717222222",151184.0,IN,22.8,83.0,"22.8, 83.0"
2002,sl,"erode, tamil nadu, indija",11.340833,77.717222,"11.340833333, 77.717222222",151184.0,IN,22.8,83.0,"22.8, 83.0"
2003,ta,ஈரோடு,11.340833,77.717222,"11.340833333, 77.717222222",151184.0,IN,22.8,83.0,"22.8, 83.0"
2004,th,เอโรด,11.340833,77.717222,"11.340833333, 77.717222222",151184.0,IN,22.8,83.0,"22.8, 83.0"
...,...,...,...,...,...,...,...,...,...,...
2095,ca,taixkent,41.300000,69.266667,"41.3, 69.266666666",2424100.0,UZ,41.0,66.0,"41.0, 66.0"
2096,ckb,تاشکەند,41.300000,69.266667,"41.3, 69.266666666",2424100.0,UZ,41.0,66.0,"41.0, 66.0"
2097,de,taschkent,41.300000,69.266667,"41.3, 69.266666666",2424100.0,UZ,41.0,66.0,"41.0, 66.0"
2098,el,τασκένδη,41.300000,69.266667,"41.3, 69.266666666",2424100.0,UZ,41.0,66.0,"41.0, 66.0"


In [100]:
df_e.to_csv('./data/location_ref/city_multilingu_citycoord_countrycode_countrycoord.csv')